In [ ]:
#@title 
catalogueEndpoint = 'http://qcmms-cat.spacebel.be/eo-catalog'
lpProductIdentifier='ProductIdentifierToBeReplaced'
lpParentIdentifier= 'ParentIdentifierToBeReplaced'
debug = False
#@title Basic functions
import json, requests
import pandas as pd
import math
import matplotlib.pyplot as plt

from IPython.display import display, HTML, Math, Latex
import ipywidgets as widgets

lpUrl=catalogueEndpoint + '/'+ 'series'+'/' + lpParentIdentifier + '/datasets/'+lpProductIdentifier
ipUrl=''
lpMd={}
platforms=[]
fgNbr = 0
references=[]
geometric_resolution=10

processingLevelDefs=["1C","L1TP","L1GT","L2A", "L2"]
maxRecords = 10

pd.set_option('display.html.table_schema', True)

def getMetric(metric,feature):
    result={}
    metrics=feature['properties']['productInformation']['qualityInformation']['qualityIndicators']
    lenMetrics= len(metrics)
    for i in range(lenMetrics):
      if (metrics[i]['isMeasurementOf'] == metric): 
        result=metrics[i]
    if (result == {} and debug):
      print("metric ", metric,"not found in ", feature['properties']['identifier'] )
    return result

def getProcessingLevelIndex(processingLevel):
  index=-1
  for i in range(len(processingLevelDefs)):
    if (processingLevelDefs[i] == processingLevel ): 
      index=i
  return index

def generateFigureLabel(fgNbr,label):
  lfgNbr=fgNbr+1
  display(widgets.HTML(
    value='<i>Figure '+ str(lfgNbr) +': ' + label+'</i>'
    ))
  return lfgNbr

def getFigureLabel(fgNbr,label):
  return 'Figure '+ str(fgNbr) +': ' + label

tableNbr=0
def generateTableLabel(tableNbr,label):
  lNbr=tableNbr+1
  display(widgets.HTML(
    value='<i>Table '+ str(lNbr) +': ' + label+'</i>'
    ))
  return lNbr

def displayImage(url):
  display(widgets.HTML(
  value= '<img width="500" height="500" src="'+url+'">' 
    ))
#@title Retrieving Land Product
def getLandProduct():
  querystring={}
  nbLp=0
  lpMd={}
  #print(lpUrl)
  response = requests.get(lpUrl, params=querystring)
  pyresult = json.loads(response.text)
  try:
    nbLp  = len(pyresult['features'])
  except KeyError:
    pass
  if (nbLp == 0):
    print("Land Product " + lpProductIdentifier + "("+ lpParentIdentifier +") is not found" )
  else:
    lpMd = pyresult['features'][0]
  return lpMd

def cvtRasterCoding(rasterCodings):
  res=''
  for i in range(len(rasterCodings)):
    res=res+cvtSingleRasterCoding(rasterCodings[i])
    res=res + '\\n'
  return res

def cvtSingleRasterCoding(rasterCoding):
  res=''
  try:
    min=toString(rasterCoding['min'])
    max=toString(rasterCoding['max'])
    name=toString(rasterCoding['name'])
    if (min == max):
	    res=min+': ' + name
    else:
      res=min + '-' + max  + ': ' + name
  except KeyError:
    pass
  return res

def toString(value):
  if isinstance(value,int):
    return str(value)
  elif isinstance(value,dict):
    return json.dumps(value)
  else:
    return value

def getLpAttribute(lpMdProperties,attribute1,attribute2):
  res=''
  try:
    if (attribute2 == 'raster_coding'):
      res=cvtRasterCoding(lpMdProperties[attribute1][attribute2])
    elif (attribute2 != ''):
      res=lpMdProperties[attribute1][attribute2]
      if isinstance(res, list):
        res=",".join(map(str,res))
    else:
      res=lpMdProperties[attribute1]
  except KeyError:
        pass
  return res 
lpMd=getLandProduct()
#@title Retrieving Related Image Products
def initImageProductsUrl():
  url=''
  lp=getLandProduct()
  if lp != {}:
    try:
      url=lp['properties']['links']['via'][0]['href']
    except KeyError:
      print('Missing via links to Image Products the Land Product') 
  return url
ipUrl=initImageProductsUrl()
#@title Retrieving Related Platforms
def initPlatforms(lpMd):
  nb_platforms=len(lpMd['properties']['acquisitionInformation'])
  lplatforms= [''] * nb_platforms    
  for i in range(len(lpMd['properties']['acquisitionInformation'])):
      lplatforms[i]=lpMd['properties']['acquisitionInformation'][i]['platform']['platformShortName']
  return lplatforms
platforms=initPlatforms(lpMd)

def harmonized():
# nbr of platforms is used to decide if we are in multisensor UC.
# if yes there some additional information for the harmonization.
  if (len(platforms) > 1):
    return True;
  else:
    return False;


def countImageProductByMetric(metric,metricValue):
   # metric can take values
   # degradedDataPercentageMetric		
   # degradedAncillaryDataPercentageMetric		
   # formatCorrectnessMetric
   # generalQualityMetric		
   # geometricQualityMetric		
   # radiometricQualityMetric		
   # sensorQualityMetric	
   # feasibilityControlMetric		
   # deliveryControlMetric		
   # ordinaryControlMetric		
   # detailedControlMetric
  querystring={}
  querystring['maximumRecords'] = 1
  if metric != '':
    querystring['specificationTitle'] = metric
    querystring['degree'] = metricValue
  response = requests.get(ipUrl, params=querystring)
  pyresult= json.loads(response.text)
  return pyresult['totalResults']
#
# build status for each steps
#
#
# return True if the metric is associated to LP 
# check if metric contains lp
def lpMetric(metric):
  if metric.lower().rfind('lp') == -1:
    return False
  else:
    return True

def  getLpMetricStatus(metricName):
  metricPrefix='http://qcmms.esa.int/quality-indicators/#'
  metric=getMetric(metricPrefix+metricName,lpMd)
  #print("metric id", metricPrefix+metricName)
  #print("lpMd",lpMd)
  #print("metric", metric)
  if metric != {}:
    executed=True
  else:
    executed=False
  applicable=True
  if metricName == 'lpInterpretationMetric' and harmonized(): 
    applicable=False
  
  nbPass=0
  nbFailed=0
  try:
    if metric['value']:
      nbPass=1
    else:
      nbFailed=1
  except KeyError:
    pass
  
  result={'Applicable': applicable, 'Executed' : executed, 'PASS': nbPass, 'FAILED' : nbFailed }
  #print("metric name", metricName, result)
  return result

def getMetricStatus(metric):
  if not lpMetric(metric):
    nbPass=countImageProductByMetric(metric,'true')
    nbFailed=countImageProductByMetric(metric,'false')
    applicable=True
    if not harmonized() and metric == 'harmonizationControlMetric':
      applicable=False
    
    if nbPass == 0 and nbFailed == 0:
      executed=False
    else:
      executed=True
    return {'Applicable': applicable, 'Executed' : executed, 'PASS': nbPass, 'FAILED' : nbFailed }
  else:
    return getLpMetricStatus(metric)


MetricsStatusVct=[
                  {'Applicable': True, 'Executed' : True, 'PASS': 1, 'FAILED' :0 },
                  getMetricStatus('feasibilityControlMetric'),
                  getMetricStatus('deliveryControlMetric'),
                  getMetricStatus('ordinaryControlMetric'),
                  getMetricStatus('detailedControlMetric'),
                  getMetricStatus('harmonizationControlMetric'),
                  getMetricStatus('ipForLpInformationMetric'),
                  getMetricStatus('lpInterpretationMetric'),
                  getMetricStatus('lpMetadataControlMetric'),
                  getMetricStatus('lpOrdinaryControlMetric'),
                  getMetricStatus('lpThematicValidationMetric')
                 ]
IndexFeasibility=1
IndexDelivery=2
IndexOrdinary=3
IndexDetailed=4
IndexHarmonization=5
IndexIpForLp=6
IndexLpInterpretation=7
IndexLpMetadata=8
IndexLpOrdinary=9
IndexLpThematic=10
#MetricsStatusVct[IndexFeasibility]['Executed']=False
#MetricsStatusVct[IndexDelivery]['Applicable']=False


def alreadyExecuted(index, printMessage):
  if not MetricsStatusVct[index]['Applicable']:
    if printMessage:
      print("This step is not applicable.")
    return False
  if not MetricsStatusVct[index]['Executed']:
    if printMessage:
      print("This step has not yet been executed.")
    return False
  return True
#
# compute the nbr of PASS/FAILED  for overall assessment and to decide if a step has already been executed.
#

IPsMetricsStatusVct=[
                     {'Metric' :'1. FeasibilityControl',  'Value' : 'PASS' ,  'Nbr' : MetricsStatusVct[IndexFeasibility]['PASS']},	
                     {'Metric' :'1. FeasibilityControl',  'Value' : 'FAILED' ,'Nbr' : MetricsStatusVct[IndexFeasibility]['FAILED'] },
                     {'Metric' :'2. DeliveryControl',     'Value' : 'PASS' ,  'Nbr' : MetricsStatusVct[IndexDelivery]['PASS'] },	
                     {'Metric' :'2. DeliveryControl',     'Value' : 'FAILED' ,'Nbr' : MetricsStatusVct[IndexDelivery]['FAILED'] },
                     {'Metric' :'3a. OrdinaryControl',    'Value' : 'PASS' ,  'Nbr' : MetricsStatusVct[IndexOrdinary]['PASS'] },
                     {'Metric' :'3a. OrdinaryControl',    'Value' : 'FAILED' ,'Nbr' : MetricsStatusVct[IndexOrdinary]['FAILED'] },	
                     {'Metric' :'3b. DetailedControl',    'Value' : 'PASS' ,  'Nbr' : MetricsStatusVct[IndexDetailed]['PASS'] },	
                     {'Metric' :'3b. DetailedControl',    'Value' : 'FAILED' ,'Nbr' : MetricsStatusVct[IndexDetailed]['FAILED'] }	]
if MetricsStatusVct[IndexHarmonization]['Applicable']:
  IPsMetricsStatusVct.append({'Metric' :'4a. HarmonizationControl',    'Value' : 'PASS' ,  'Nbr' : MetricsStatusVct[IndexHarmonization]['PASS'] })
  IPsMetricsStatusVct.append({'Metric' :'4a. HarmonizationControl',    'Value' : 'FAILED' ,  'Nbr' : MetricsStatusVct[IndexHarmonization]['FAILED']  })

def generateDefinitions():
  def0= '<tr><td>'+'Circular Error at the 90th percentile' + '</td><td>'+ 'A minimum of 90 percent of the points are included in the stated CE90 circle.' + '</td></tr>'
  def0a= '<tr><td>'+'EEA reference grid' + '</td><td>'+ 'The grid is based on the recommendation at the 1st European Workshop on Reference Grids in 2003 and later INSPIRE geographical grid systems. A short specification can be downloaded <a href="https://www.eea.europa.eu/data-and-maps/data/eea-reference-grids-2/about-the-eea-reference-grid/eea_reference_grid_v1.pdf/at_download/filehere eea_reference_grid_v1.pdf">here.</a>  ' + '</td></tr>'
  def1= '<tr><td>'+'Image Product' + '</td><td>'+ 'Satellite image data with metadata used as input to Land Mapping Production.' + '</td></tr>'
  def1a= '<tr><td>'+'Kappa' + '</td><td>'+ "Coefficient that is generated from a statistical test to evaluate the accuracy of a classification. Kappa evaluates how well the classification performed as compared to just randomly assigning values, i.e. did the classification do better than random."
  def1b =  "The Kappa Coefficient can range from -1 to 1. A value of 0 indicated that the classification is no better than a random classification. A negative number indicates the classification is significantly worse than random."
  def1c = " A value close to 1 indicates that the classification is significantly better than random. It is expressed as a percentage." + '</td></tr>'
  def2= '<tr><td>'+'Land Product'+ '</td><td>' + 'Land Mapping or Monitoring Product as e.g. Land Cover Map.'+'</td></tr>' 
  def3= '<tr><td>'+'Land Product Specification Table'+'</td><td>'+ 'Set of parameters that defines the Land Product and the required Image Products.'+'</td></tr>' 
  def3a= '<tr><td>'+'Overall Accuracy'+'</td><td>'+ 'Classification quality indicator that tells us out of all of the reference sites what proportion were mapped correctly. The overall accuracy is usually expressed as a percent, with 100% accuracy being a perfect classification where all reference sites were classified correctly.'+'</td></tr>' 
  def3b= '<tr><td>'+"Pearson's r" + '</td><td>'+ "Statistic that measures linear correlation between two variables, the mapped values of a specific phenomena and the associated references." + '</td></tr>'
  def3c= '<tr><td>'+'Producers Accuracy' + '</td><td>'+ "Map accuracy from the point of view of the map producer (the land product producer). This is how often are real features on the ground correctly shown on the classified map or the probability that a certain land cover of an area on the ground is classified as such. The Producer's Accuracy is complement of the Omission Error, Producer's Accuracy = 100%-Omission Error. Errors of omission refer to reference sites that were left out (or omitted) from the correct class in the classified map." + '</td></tr>'
 
  def4= '<tr><td>'+'Root Mean Square Error' +'</td><td>'+  'The standard deviation of the residuals (prediction errors). Residuals are a measure of how far from the regression line data points are; RMSE is a measure of how spread out these residuals are. In other words, it tells you how concentrated the data is around the line of best fit.' +'</td></tr>' 
  def5= '<tr><td>'+'Thematic accuracy'+'</td><td>'+ 'Same definition as overall accuracy'+'</td></tr>' 
  def6= '<tr><td>'+'Users Accuracy'+'</td><td>'+ "The User's Accuracy is the accuracy from the point of view of a map user, not the map producer. The User's accuracy essentially tells use how often the class on the map will actually be present on the ground." 
  def6a= " This is referred to as reliability. The User's Accuracy is complement of the Commission Error, User's Accuracy = 100%-Commission Error. Commission errors are calculated by reviewing the classified sites for incorrect classifications."
  def6b= " The User's Accuracy is calculating by taking the total number of correct classifications for a particular class and dividing it by the row total. " + '</td></tr>' 
  definitions= '<table  border=1 width=80%><col width=15%><col width=50%>' + def0 + def0a + def1 + def1a + def1b + def1c + def2 + def3 + def3a + def3b+ def3c+ def4 + def5 + def6 + def6a + def6b +'</table>'
  display(widgets.HTML(
    value=definitions
    ))
#generateDefinitions()
def generateAbbreviations():
  def0a= '<tr><td>'+'AOI' + '</td><td>'+ 'Area Of Interest' + '</td></tr>'
  def0b= '<tr><td>'+'CE90' + '</td><td>'+ 'Circular Error at the 90th percentile' + '</td></tr>'
  def0c= '<tr><td>'+'CRS' + '</td><td>'+ 'Coordinate Reference Systems' + '</td></tr>'
  def0d= '<tr><td>'+'EEA' + '</td><td>'+ 'European Environment Agency' + '</td></tr>'
  def1= '<tr><td>'+'EO' + '</td><td>'+ 'Earth Observation' + '</td></tr>'
  def2= '<tr><td>'+'HR' + '</td><td>'+ 'High Resolution' + '</td></tr>'
  def3= '<tr><td>'+'HRL' + '</td><td>'+ 'High Resolution Layer' + '</td></tr>'
  def4= '<tr><td>'+'IP' + '</td><td>'+ 'Image Product' + '</td></tr>'
  def5= '<tr><td>'+'INSPIRE' + '</td><td>'+ 'Infrastructure for Spatial Information in Europe' + '</td></tr>'
  def6= '<tr><td>'+'LCCM' + '</td><td>'+ 'Land Cover Continuous Monitoring' + '</td></tr>'
  def7= '<tr><td>'+'LP'+  '</td><td>' + 'Land Product' + '</td></tr>'
  def8a= '<tr><td>'+'MAE'+  '</td><td>' + 'Mean Absolute Error' + '</td></tr>'
  def8b= '<tr><td>'+'MMU'+  '</td><td>' + 'Minimum Mapping Unit' + '</td></tr>'
  def8c= '<tr><td>'+'MSE'+  '</td><td>' + 'Mean Square Error' + '</td></tr>'
  def9= '<tr><td>'+'RMSE'+  '</td><td>' + 'Root Mean Square Error' + '</td></tr>'
  def9a= '<tr><td>'+'u8'+  '</td><td>' + 'Unsigned 8-bit' + '</td></tr>'
  def10= '<tr><td>'+'VHR'+  '</td><td>' + 'Very High Resolution' + '</td></tr>'
  definitions= '<table border=0 width=80%>' + def0a+ def0b+ def0c+ def0d + def1 + def2 + def3 + def4 + def5 + def6 + def7 + def8a + def8b + def8c+ def9+ def9a+ def10+'</table>'
  display(widgets.HTML(
    value=definitions
    ))

#@title Generate Land Product Definition
def generate_lp_definition_report(tableNbr):
  table_title =['Parameter','Value/Description']
  
  table_report = []
  lpMd = getLandProduct()
  if(lpMd != {}):
    abstract=lpMd['properties']['abstract']
    
    display(widgets.HTML(
       value= abstract ))
    display(widgets.HTML(value='<br>'))
    tableNbr=generateTableLabel(tableNbr,'Land Product Specification Table')
    geometric_resolution=getLpAttribute(lpMd['properties'],'additionalAttributes','geometric_resolution')

    table_report=[
      ['Product',getLpAttribute(lpMd['properties'],'title','')],
      ['Product focus',getLpAttribute(lpMd['properties'],'additionalAttributes','product_focus')],   
      ['Geometric resolution',str(getLpAttribute(lpMd['properties'],'additionalAttributes','geometric_resolution'))+'m'],
      ['Target CRS',getLpAttribute(lpMd['properties'],'additionalAttributes','crs')], 
      ['Grid',getLpAttribute(lpMd['properties'],'additionalAttributes','grid')], 
      ['Geometric accuracy',str(getLpAttribute(lpMd['properties'],'additionalAttributes','geometric_accuracy'))+' pixel'], 
      ['Thematic accuracy',str(getLpAttribute(lpMd['properties'],'additionalAttributes','thematic_accuracy'))+'%'], 
      ['MMU',str(getLpAttribute(lpMd['properties'],'additionalAttributes','mmu_pixels'))+' pixel'],  
      ['Necessary attributes',getLpAttribute(lpMd['properties'],'additionalAttributes','necessary_attributes')],
      ['Raster coding',getLpAttribute(lpMd['properties'],'additionalAttributes','raster_coding')],
      ['Data type',getLpAttribute(lpMd['properties'],'additionalAttributes','data_type')],
      ['Coverage area',getLpAttribute(lpMd['properties'],'additionalAttributes','coverage_area')],
      ['Reference year',getLpAttribute(lpMd['properties'],'additionalAttributes','reference_year')],
      ['Seasonal window','months '+getLpAttribute(lpMd['properties'],'additionalAttributes','seasonal_window')],
      ['Temporal focus',getLpAttribute(lpMd['properties'],'additionalAttributes','temporal_focus')]          
    ]
    table_index = ['','','','','','','','','','','','','','','']
    pd_report = pd.DataFrame(table_report , index=table_index,columns = table_title)
    dfStyler=pd_report.reset_index(drop=True).style.set_properties( **{'text-align': 'left'})

    #dfStyler = pd_report.style.set_properties(**{'text-align': 'left'})
    dfStyler.set_table_styles([dict(selector='th', props=[('text-align', 'left')])])
    
    #pd_report.head()
    pretty_print(pd_report,'')
    #display(pd_report)
    return  {"tableNbr" : tableNbr,
             "geometric_resolution" : geometric_resolution
            }
   

#@title Generate Feasibility Control Report
def generate_feasibility_control_report(platform):
  querystring = {}
  querystring['maximumRecords'] = maxRecords
  querystring['platform'] = platform
  table_title =['Format correctness','General quality', 'Geometric quality', 'Radiometric quality', 'Sensor quality', 'Total']
  metrics=['formatCorrectnessMetric','generalQualityMetric','geometricQualityMetric','radiometricQualityMetric','sensorQualityMetric','feasibilityControlMetric']
  degrees=['true','false']
  table_report=[ [0]*len(table_title) for i in range(len(degrees))]
  for i in range(len(degrees)):
    querystring['degree'] = degrees[i]
    for j in range (len(metrics)):
      querystring['specificationTitle'] = metrics[j]
      response = requests.get(ipUrl, params=querystring)
      table_report[i][j]=json.loads(response.text)['totalResults'] 
  pd_report = pd.DataFrame(table_report , index = ['TRUE', 'FALSE'], columns = table_title)
  display(pd_report)
  display(widgets.HTML(value='<br>'))


#@title Generate Frequency Plot of Cloud Cover

def generateHist(fgNbr,platform):
  querystring = {}
  cloudCvPctVt=[]
  querystring['maximumRecords'] = maxRecords
  querystring['platform'] = platform
  totalResults=1
  startIndex=1
  while (startIndex<=totalResults):
    querystring['startRecord'] = startIndex
    response = requests.get(ipUrl, params=querystring)
    pyresult= json.loads(response.text)
    lenFeatures= len(pyresult['features'])
    startIndex=startIndex+maxRecords
    totalResults=pyresult['totalResults']
    for i in range(lenFeatures):
      try:
        cloudCvPctVt.append(pyresult['features'][i] ['properties']['productInformation']['cloudCover'])
       # print (pyresult['features'][i] ['properties']['productInformation']['cloudCover'])
      except KeyError:
        pass
  fgNbr=fgNbr+1
  df = pd.DataFrame({getFigureLabel(fgNbr,"Frequency plot of the cloud cover % for " + platform )+' \n': cloudCvPctVt}) 
  hist = df.hist(bins=100)
  hist=hist[0]
  for x in hist:
     x.set_xlabel("Cloud cover percentage", labelpad=20, weight='bold', size=12)
     x.set_ylabel("Number of images", labelpad=20, weight='bold', size=12)
  
  display(widgets.HTML(value='<br>'))
  
  return fgNbr

#generateHist(fgNbr,'Sentinel-2')
#@title Generate Delivery Control Report
def generate_delivery_control_report(platforms):
  table_index = ["" for i in range(len(platforms)+1)]
  table_title = ['TRUE','FALSE', 'WAITING', 'IN_PROGRESS']
  metric="http://qcmms.esa.int/quality-indicators/#deliveryControlMetric"
  for i in range(len(platforms)):
    table_index[i]=platforms[i]
  table_index[len(platforms)]='Total'

  def fillReportTable(iplat, deliveryControlMetric,table):

    if deliveryControlMetric['complete']:
        table[iplat][0]=table[iplat][0]+1
    else:
        table[iplat][1]=table[iplat][1]+1

    if (deliveryControlMetric['status'] == "WAITING"):
        table[iplat][2]=table[iplat][2]+1
    elif (deliveryControlMetric['status'] == "IN_PROGRESS"):
        table[iplat][3]=table[iplat][3]+1
   
  def completeReportTable(table):
 
    nbplat=len(platforms) 
    for i in range(len(table_index)):     
      for j in range(nbplat):   
        table[nbplat][i]=table[nbplat][i]+table[j][i]

  table_report=[ [0]*4 for i in range(len(platforms)+1)]
  for indexPlatform  in range(len(platforms)):
    querystring={}
    querystring['maximumRecords'] = maxRecords
    totalResults=1
    startIndex=1
 
    while (startIndex<=totalResults):
      querystring['startRecord'] = startIndex
      querystring['platform'] = platforms[indexPlatform]
      response = requests.get(ipUrl, params=querystring)
      pyresult= json.loads(response.text)
      lenFeatures= len(pyresult['features'])
      startIndex=startIndex+maxRecords
      totalResults=pyresult['totalResults']
      for i in range(lenFeatures):       
        deliveryControlMetric=getMetric(metric,pyresult['features'][i])
        if (deliveryControlMetric != {}):
          fillReportTable(indexPlatform, deliveryControlMetric, table_report)

  completeReportTable(table_report)

  pd_report = pd.DataFrame(table_report ,index = table_index, columns = table_title)
  display(pd_report)   
  display(widgets.HTML(value='<br>')) 
#generate_delivery_control_report(platforms)
#generate_delivery_control_report('Sentinel-2')
#@title Generate Ordinary Control Report
def generate_ordinary_control_report(platform):
  
  def initTableReport():
    table=[ ["FALSE",0,0,0,0,0] for i in range(len(procLevelDefs)*2)]
    for i in range(len(procLevelDefs)):
      table[i*2][0]="TRUE"
    return table
  
  def getAttribute(attribute, metric):
    result={'present' : True,'value':True}
    try:
      result['value']=metric[attribute]
    except KeyError:
      result['present'] = False
    return result
  
  def getMetricValue(metricValue):
    result={'present' : True,'value':True}
    try:
      result['value']=metricValue
    except KeyError:
      result['present'] = False
    return result

  def getRaster(ordinaryControlMetric):
    result={}
    try:
      result=ordinaryControlMetric['harmonized']['tile'][0]['raster']
    except KeyError:
      pass
    return result


  def fillReportByLevel(level,ordinaryControlMetric):
    try:
      # compute table from rastsers complete to end
      table_row=[{'present' : False,'value':True},
                 {'present' : False,'value':True},
                 {'present' : False,'value':True},
                 {'present' : False,'value':True},
                 {'present' : False,'value':True},
                 {'present' : False,'value':True}]
      # prepare working row to avoid list of if else for all columns
      if (level=='harmonized'):
        raster=getRaster(ordinaryControlMetric)
        if (raster != {}):
          table_row[0] = getAttribute('rastersComplete', raster) 
          table_row[1] = getAttribute('rastersRead', raster)     
          table_row[2] = getAttribute('calibrationMetadata', raster)     
          table_row[3] = getAttribute('metadataRead', raster)  
          table_row[4] = getMetricValue(ordinaryControlMetric['value']) 
      else:
        table_row[0] = getAttribute('rastersComplete', ordinaryControlMetric[level]) 
        table_row[1] = getAttribute('rastersRead', ordinaryControlMetric[level])     
        table_row[2] = getAttribute('calibrationMetadata', ordinaryControlMetric[level])     
        table_row[3] = getAttribute('metadataRead', ordinaryControlMetric[level])  
        table_row[4] = getMetricValue(ordinaryControlMetric['value']) 
  #    table_row[5] = getAttribute('rows', ordinaryControlMetric[level])
     
      #table_index=processingLevelIndex*2
      if (level=='level1'):
        table_index=0
      elif (level=='level2'):
        table_index=2
      else:
        table_index=4
      # for each columns, the row is determined by the processing level &
      # and boolean value of the corresponding attribute
      # the second column has already been filled by the table initialisation
      # note that the titles and first column are not part of the table
      # are passed at the dataframe construction

      # for each column
      for i in range(len(table_row)):
        if (table_row[i]['present']):
          # compute table index = table row nbr
          # second line of boolean is false 
          if (table_row[i]['value']):
            ti = table_index
          else:
            ti = table_index+1
          # compute table column, first columns is fixed.
          tc=i+1 
          table_report[ti][tc]=table_report[ti][tc]+1
    except KeyError:
      pass

  def fillReportTable(processingLevel, ordinaryControlMetric):
    processingLevelIndex=getProcessingLevelIndex(processingLevel)
    #print("level ",processingLevel, "index ",processingLevelIndex)
    if (processingLevelIndex != -1):
      fillReportByLevel('level1',ordinaryControlMetric)
      fillReportByLevel('level2',ordinaryControlMetric)
      if harmonized():
         fillReportByLevel('harmonized',ordinaryControlMetric)
  
  procLevelDefs=['L1', 'L2']
  table_index = [procLevelDefs[0],'', procLevelDefs[1],'']
  if harmonized():    
    procLevelDefs=['L1', 'L2','LH'] 
    table_index = [procLevelDefs[0],'', procLevelDefs[1],'', procLevelDefs[2],'']
  
  table_title = ['Eval.','Rasters complete', 'Rasters read', 'Calibration metadata', 'Metadata read','Total' ]
  

  
  metric="http://qcmms.esa.int/quality-indicators/#ordinaryControlMetric"

  table_report=initTableReport()  
  querystring={}
  querystring['maximumRecords'] = maxRecords
  querystring['platform'] = platform
  totalResults=1
  startIndex=1
  while (startIndex<=totalResults):
    querystring['startRecord'] = startIndex
    response = requests.get(ipUrl, params=querystring)
    pyresult= json.loads(response.text)
    lenFeatures= len(pyresult['features'])
    startIndex=startIndex+maxRecords
    totalResults=pyresult['totalResults']
    for i in range(lenFeatures):
      processingLevel=pyresult['features'][i]['properties']['productInformation']['processingLevel']
      ordinaryControlMetric=getMetric(metric,pyresult['features'][i])
      if (ordinaryControlMetric != {}):
        fillReportTable(processingLevel, ordinaryControlMetric)
  #print("end report")
  
  pd_report = pd.DataFrame(table_report ,index = table_index, columns = table_title)
  display(pd_report)
  display(widgets.HTML(value='<br>'))
#generate_ordinary_control_report("LANDSAT-8")
#generate_ordinary_control_report("Sentinel-2")
#@title Generate Detailed Control Report
def generate_detailed_control_report(platform):
  
  def fillReportTableCell(tc,detailedControlMetric, attributeName, metricId):
    ti=1
    try:
      vect=detailedControlMetric[attributeName]
      for i in range(len(vect)):
        if ( (vect[i]["id"]==metricId) and (vect[i]["mask"]!="" )):
          ti=0
    except KeyError:
      pass
    table_report[ti][tc]=table_report[ti][tc]+1
    #print(ti, tc, table_report[ti][tc] )

  def fillReportTableLastCell(tc,detailedControlMetric):
    ti=1
    try:
      if (detailedControlMetric['value']):
        ti=0
    except KeyError:
      pass
    table_report[ti][tc]=table_report[ti][tc]+1
    #print(ti, tc, table_report[ti][tc] )

  def fillReportTable(detailedControlMetric):
    fillReportTableCell(0,detailedControlMetric, "cloudCover", "http://qcmms.esa.int/detailed_control#CLOUD_COVER")
    fillReportTableCell(1,detailedControlMetric, "cloudCover", "http://qcmms.esa.int/detailed_control#ALTERNATE_CLOUD_COVER")
    fillReportTableCell(2,detailedControlMetric, "geometry", "http://qcmms.esa.int/detailed_control#GEOMETRY")
    fillReportTableCell(3,detailedControlMetric, "validPixels", "http://qcmms.esa.int/detailed_control#VALID_PIXELS")
    fillReportTableLastCell(4,detailedControlMetric)

  table_title = ['Cloud cover layer','Cloud cover alternative layer', 'Geometry val.layer', 'Valid pixel metadata', 'TOTAL' ]
  table_index = ['TRUE','FALSE']
  metric="http://qcmms.esa.int/quality-indicators/#detailedControlMetric"
  table_report=[ [0,0,0,0,0],  [0,0,0,0,0]]  

  querystring={}
  querystring['maximumRecords'] = maxRecords
  querystring['platform'] = platform
  totalResults=1
  startIndex=1
  while (startIndex<=totalResults):
    querystring['startRecord'] = startIndex   
    response = requests.get(ipUrl, params=querystring)
    pyresult= json.loads(response.text)
    lenFeatures= len(pyresult['features'])
    startIndex=startIndex+maxRecords
    totalResults=pyresult['totalResults']
    for i in range(lenFeatures):
      detailedControlMetric=getMetric(metric,pyresult['features'][i])
      if (detailedControlMetric != {}):
        fillReportTable(detailedControlMetric)
  pd_report = pd.DataFrame(table_report ,index = table_index, columns = table_title)
  display(pd_report)   
  
  display(widgets.HTML(value='<br>'))
#generate_detailed_control_report()
#@title Generate Detailed Control Stat Report
def generate_detailed_control_stat_report(platform):
  
  def fillReportTableCell(tc,detailedControlMetric, attributeName1,attributeName2,metricId):
    
    try:
      vect=detailedControlMetric[attributeName1]
      for i in range(len(vect)):
        if (vect[i]["id"]==metricId):
          table_report[tc]=table_report[tc]+vect[i][attributeName2]
          table_number[tc]=table_number[tc]+1
    except KeyError:
      pass

  def fillReportTableCellDiff(tc,detailedControlMetric,attributeName1, attributeName2,  metricId,diffMaxLimit):
    try:
      vect=detailedControlMetric[attributeName1]
      for i in range(len(vect)):
        if (vect[i]["id"]==metricId):
          if vect[i][attributeName2] > diffMaxLimit:
            table_report[tc]=table_report[tc]+1          
    except KeyError:
      pass

  def computeMean():
    for i in range(len(table_number)):
      if (table_number[i] != 0):
        table_report[i] = table_report[i] / table_number[i]

  def fillReportTable(detailedControlMetric):
    fillReportTableCell(0,detailedControlMetric, "cloudCover",  "cloudPct", "http://qcmms.esa.int/detailed_control#CLOUD_COVER")
    fillReportTableCell(1,detailedControlMetric, "cloudCover",  "cloudPct", "http://qcmms.esa.int/detailed_control#ALTERNATE_CLOUD_COVER")
    fillReportTableCell(2,detailedControlMetric, "geometry",    "rmseX",    "http://qcmms.esa.int/detailed_control#GEOMETRY")
    fillReportTableCell(3,detailedControlMetric, "geometry",    "rmseY",    "http://qcmms.esa.int/detailed_control#GEOMETRY")
    fillReportTableCell(6,detailedControlMetric, "validPixels", "validPct",  "http://qcmms.esa.int/detailed_control#VALID_PIXELS")
    fillReportTableCellDiff(4,detailedControlMetric, "geometry", "diffXmax",  "http://qcmms.esa.int/detailed_control#GEOMETRY",diffMaxLimit)
    fillReportTableCellDiff(5,detailedControlMetric, "geometry", "diffYmax",  "http://qcmms.esa.int/detailed_control#GEOMETRY",diffMaxLimit)
    
  diffMaxLimit = geometric_resolution / 2 
  dmlstr=str(diffMaxLimit)
  table_index = ['']
  table_title = ['Mean\\nCloud cover\\n[%]','Mean\\nCloud alternative\\n[%]', 'Mean\\nGeometry RmseX\\n[m]', 'Mean\\nGeometry RmseY\\n[m]','Nb > '+ dmlstr +'\\nGeometry Diffx Max', 'Nb > '+ dmlstr +'\\nDiffy Max' ,'Mean\\nValid Pixels\\n[%]' ]
  table_number =[   0,               0,                 0,                 0,                1,                1,           0]  # used to compute mean when required.
  metric="http://qcmms.esa.int/quality-indicators/#detailedControlMetric"
  
  table_report=[0,0,0,0,0,0,0]  
  querystring={}
  querystring['maximumRecords'] = maxRecords
  querystring['platform'] = platform
  totalResults=1
  startIndex=1
  while (startIndex<=totalResults):
    querystring['startRecord'] = startIndex
    response = requests.get(ipUrl, params=querystring)
    pyresult= json.loads(response.text)
    lenFeatures= len(pyresult['features'])
    startIndex=startIndex+maxRecords
    totalResults=pyresult['totalResults']
    for i in range(lenFeatures):
      detailedControlMetric=getMetric(metric,pyresult['features'][i])
      if (detailedControlMetric != {}):
        fillReportTable(detailedControlMetric)
  computeMean()
  table_report2=[table_report]
  pd_report = pd.DataFrame(table_report2,index = table_index, columns = table_title)
  #display(pd_report)  
  pretty_print(pd_report,'')
  display(widgets.HTML(value='<br>')) 
# generate_detailed_control_stat_report()
#@title Plot CE
def generate_Plot_CE(fgNbr,platform):

  def computeCircularError(percentile):
    # compute the index in the sorted array to have the required percentage (percentile)  of points inside the circle of center 0,0 and radius = that distance 
    index=math.ceil((pd_report.size/3)*(percentile/100.0))
    # return the computed distance (column=2) 
    return pd_report.iat[index-1,2]

  def plotCE(ce,title):
    x_vec=pd_report['diffXmax'].values.tolist()
    y_vec=pd_report['diffYmax'].values.tolist()
     
    if (( len(x_vec)==0 ) or ( len(y_vec)==0 )):
      print("no data available for the plot (diff Xmax, diff Ymax)")
    else:
      plt.scatter(x_vec,y_vec)
      fig=plt.gcf()
      ax=fig.gca()
      ax.axis('equal')
      #plt.ioff()
      #plt.hold(False)
      #print("interactive ",plt.isinteractive())
      title=title+' \n\n ' + 'CE'+str(percentile) + " = " + str(round(ce,1))+"m"
      plt.title(title) 
      #  plt.title(title + ' 
      #  CE'+str(percentile) + " = " + str(round(ce,1)))
      plt.xlabel("x-offset [m]")
      plt.ylabel("y-offset [m]")
      ce_circle=plt.Circle((0,0),ce,edgecolor='r',fill=False)
      ax.add_artist(ce_circle)
      plt.show()

  def distFromOrigin(xMax,yMax):
    return math.sqrt(xMax*xMax+yMax*yMax)
  
  def fillReportTableCell(detailedControlMetric,attributeName,  metricId,dataAvailable):
    try:
      vect=detailedControlMetric[attributeName]
      for i in range(len(vect)):
        if (vect[i]["id"]==metricId):
          xMax=vect[i]['diffXmax']
          yMax=vect[i]['diffYmax']
          dist=distFromOrigin(xMax,yMax)
          table_report.append([xMax,yMax,dist])
          dataAvailable=True
        
    except KeyError:
      pass
    return dataAvailable

  def fillReportTable(detailedControlMetric,dataAvailable):
    return fillReportTableCell(detailedControlMetric, "geometry", "http://qcmms.esa.int/detailed_control#GEOMETRY",dataAvailable)
     
  metric="http://qcmms.esa.int/quality-indicators/#detailedControlMetric"  
  querystring={}
  querystring['maximumRecords'] = maxRecords
  querystring['platform'] = platform
  totalResults=1
  startIndex=1
  table_report=[]
  table_column=['diffXmax','diffYmax','distance']
  percentile=90
  dataAvailable = False
  while (startIndex<=totalResults):
    querystring['startRecord'] = startIndex
    
    response = requests.get(ipUrl, params=querystring)
    pyresult= json.loads(response.text)
    lenFeatures= len(pyresult['features'])
   
    
    startIndex=startIndex+maxRecords
    totalResults=pyresult['totalResults']
    if(lenFeatures > 0):
       for i in range(lenFeatures):
          detailedControlMetric=getMetric(metric,pyresult['features'][i])           
          dataAvailable=fillReportTable(detailedControlMetric, dataAvailable)
  if(dataAvailable):
     pd_report = pd.DataFrame(table_report,columns=table_column)
     pd_report.sort_values(["distance"],axis=0,ascending=[True],inplace=True)
     #display(pd_report)
     fgNbr=fgNbr+1
     gr_str=str(geometric_resolution)
     title=getFigureLabel(fgNbr,"Geometry validation results, requirement: geometry quality < 0.5 pixel of "+ gr_str +" m for " + platform )
     circularError=computeCircularError(percentile)
     plotCE(circularError, title)
     #pd_report.plot.scatter(x='diffXmax',y='diffYmax') 
  else:
    print("No "+ platform +" data available for this plot.")
  return fgNbr  
#generate_Plot_CE(0 ,'LANDSAT-8')

def generate_radiometryConsistency_control_report(platform):
#------------------------------------------------------------------------------------
  def fillReportTableCell(tc,detailedControlMetric, attributeName1,attributeName2,metricId):
    
    try:
      vect=detailedControlMetric[attributeName1]
      for i in range(len(vect)):
        if (vect[i]["id"]==metricId):
          table_report[tc]=table_report[tc]+vect[i][attributeName2]
          table_number[tc]=table_number[tc]+1
    except KeyError:
      pass

  

  def computeMean():
    for i in range(len(table_number)):
      if (table_number[i] != 0):
        table_report[i] = table_report[i] / table_number[i]

  def fillReportTable(detailedControlMetric):
    fillReportTableCell(0,detailedControlMetric, "cloudCover",  "cloudPct", "http://qcmms.esa.int/detailed_control#CLOUD_COVER")
    fillReportTableCell(1,detailedControlMetric, "cloudCover",  "cloudPct", "http://qcmms.esa.int/detailed_control#ALTERNATE_CLOUD_COVER")
    fillReportTableCell(2,detailedControlMetric, "geometry",    "rmseX",    "http://qcmms.esa.int/detailed_control#GEOMETRY")
    fillReportTableCell(3,detailedControlMetric, "geometry",    "rmseY",    "http://qcmms.esa.int/detailed_control#GEOMETRY")
    fillReportTableCell(6,detailedControlMetric, "validPixels", "validPct",  "http://qcmms.esa.int/detailed_control#VALID_PIXELS")
    fillReportTableCellDiff(4,detailedControlMetric, "geometry", "diffXmax",  "http://qcmms.esa.int/detailed_control#GEOMETRY",diffMaxLimit)
    fillReportTableCellDiff(5,detailedControlMetric, "geometry", "diffYmax",  "http://qcmms.esa.int/detailed_control#GEOMETRY",diffMaxLimit)
    
  table_index = ['']
  table_title = ['Mean\\nCloud cover','Mean\\nCloud alternative', 'Mean\\nGeometry RmseX', 'Mean\\nGeometry RmseY','Nb > 5\\nGeometry DiffX Max', 'Nb > 5\\nDiffY Max' ,'Mean\\nValid Pixels' ]
  table_number =[   0,               0,                 0,                 0,                1,                1,           0]  # used to compute mean when required.
  metric="http://qcmms.esa.int/quality-indicators/#harmonizationControlMetric"
  
  table_report=[0,0,0,0,0,0,0]  
  querystring={}
  querystring['maximumRecords'] = maxRecords
  querystring['platform'] = platform
  totalResults=1
  startIndex=1
  while (startIndex<=totalResults):
    querystring['startRecord'] = startIndex
    response = requests.get(ipUrl, params=querystring)
    pyresult= json.loads(response.text)
    lenFeatures= len(pyresult['features'])
    startIndex=startIndex+maxRecords
    totalResults=pyresult['totalResults']
    for i in range(lenFeatures):
      radiometryConsistencyControlMetric=getMetric(metric,pyresult['features'][i])
      if (detailedControlMetric != {}):
        fillReportTable(detailedControlMetric)
  computeMean()
  table_report2=[table_report]
  pd_report = pd.DataFrame(table_report2,index = table_index, columns = table_title)
  #display(pd_report)  
  pretty_print(pd_report,'')
  display(widgets.HTML(value='<br>')) 
#------------------------------------------------------------------------------------

def displayRadiometryConsistencyBarchart(fgNbr,platform):
  
  nbStat=3
  
  StatTable={}
  References=[]
  def fillMin(value,var):
    if value < var:
      var=value
    return var

  def fillMax(value,var):
    if value > var:
      var=value
    return var
  
  def fillMean(value,var):
    var=value+var
    return var
   
          
  def computeMean():
    for i in range(len(References)):
      for bd in  StatTable[References[i]['category']]:
        StatTable[References[i]['category']] [bd] ['Mean'] =StatTable[References[i]['category']] [bd] ['Mean'] / StatTable[References[i]['category']] [bd] ['Nb']

  def  fillStatTable(firstRecord,radiometryConsistencyControlMetric,References):
    bandsTempConsistency=radiometryConsistencyControlMetric['radiometryConsistency'][0]['bandsTemporalConsistency']
    if firstRecord:
      References=radiometryConsistencyControlMetric['radiometryConsistency'][0]['references']
      for i in range(len(References)): 
        StatTable[References[i]['category']]={}
        
        for j in range(len(bandsTempConsistency)): 
          StatTable[References[i]['category']][bandsTempConsistency[j]['id'] ]={}
          StatTable[References[i]['category']][bandsTempConsistency[j]['id'] ]['Min']=999999          
          StatTable[References[i]['category']][bandsTempConsistency[j]['id'] ]['Max']=-999999
          StatTable[References[i]['category']][bandsTempConsistency[j]['id'] ]['Mean']=0
          StatTable[References[i]['category']][bandsTempConsistency[j]['id'] ]['Nb']=0

    for j in range(len(bandsTempConsistency)):
       for i in range(len(References)):
         value=bandsTempConsistency[j][References[i]['category']]
         bd=bandsTempConsistency[j]['id']
         StatTable[References[i]['category']][bd] ['Min']=fillMin(value, StatTable[References[i]['category']][bd] ['Min'])
         StatTable[References[i]['category']][bd] ['Max']=fillMax(value, StatTable[References[i]['category']][bd] ['Max'])
         StatTable[References[i]['category']][bd] ['Mean']=fillMean(value,StatTable[References[i]['category']][bd] ['Mean'])
         StatTable[References[i]['category']][bd] ['Nb']= StatTable[References[i]['category']][bd] ['Nb']+1

    return References
       

  def cvtStatTableToBarChartAll(ref,figureLabel):
    nbBand=len(StatTable[ref])
    i=0
    vt=[{}]*nbBand*nbStat
    for bd in StatTable[ref]:
      for stat in StatTable[ref][bd]:
        if stat != 'Nb':
          vt[i]={figureLabel :bd,  'Stat' : stat , 'Value' : StatTable[ref][bd][stat]}
          i=i+1
    return vt

  def cvtStatTableToBarChart(ref,figureLabel):
    nbBand=len(StatTable[ref])
    i=0
    vt=[{}]*nbBand*1
    for bd in StatTable[ref]:
      for stat in StatTable[ref][bd]:
        if stat == 'Mean':
          vt[i]={figureLabel :bd,  'Stat' : stat , 'Value' : StatTable[ref][bd][stat]}
          #vt[i]={figureLabel :bd,  'Ref' : ref , 'Value' : StatTable[ref][bd][stat]}
          i=i+1
    return vt
#-------------------------------------
  querystring={}
  querystring['maximumRecords'] = maxRecords
  querystring['platform'] = platform
  metric="http://qcmms.esa.int/quality-indicators/#harmonizationControlMetric"
  totalResults=1
  startIndex=1
  firstRecord=True
  while (startIndex<=totalResults):
    querystring['startRecord'] = startIndex
    response = requests.get(ipUrl, params=querystring)
    pyresult= json.loads(response.text)
    lenFeatures= len(pyresult['features'])
    startIndex=startIndex+maxRecords
    totalResults=pyresult['totalResults']
    for i in range(lenFeatures):
      radiometryConsistencyControlMetric=getMetric(metric,pyresult['features'][i])
      if (radiometryConsistencyControlMetric != {}):
          References=fillStatTable(firstRecord,radiometryConsistencyControlMetric,References)
          firstRecord=False
         
  computeMean()
  for i in range(len(References)):
    fgNbr = fgNbr+1
    figureLabel="\n\n"+"Figure "+ str(fgNbr) + " : Temporal Consistency for "+ platform +" and reference " + References[i]['category'] + "\n\n\n"    
    vt=cvtStatTableToBarChart(References[i]['category'],figureLabel)                
    df = pd.DataFrame(vt)
    df.groupby([figureLabel,'Stat']).sum().unstack().plot(kind='bar') 
    #display(widgets.HTML(value='<br><br>'))
  return {"fgNbr":fgNbr,"references":References}
#displayRadiometryConsistencyBarchart(1,"Sentinel-2")

#------------------------------------------------------------------------------------


#@title Fit-for-Purpose
def getFitForPurpose(fgNbr):
 
  table_title = ['Min','Max', 'Gap percentage' ]
 
  metric="http://qcmms.esa.int/quality-indicators/#harmonizationControlMetric"
  lpMd=getLandProduct()
  if(lpMd != {}):
    metric=getMetric("http://qcmms.esa.int/quality-indicators/#ipForLpInformationMetric",lpMd)
    display(widgets.HTML(value='The status of the spatial coverage analysis is: <b>' + metric['fitnessForPurpose']+ '</b><br>'))
    table_index = sorted(metric['vpxCoverage'].keys())
    table_report = [[0] *  len(table_title) for i in range(len(table_index)) ]
    yi=0
    for year in table_index:
 #   for year in sorted(metric['vpxCoverage'].keys()):
      image=metric['vpxCoverage'][year]['mask']
      displayImage(image)
      fgNbr=generateFigureLabel(fgNbr,"Fit For Purpose mask year " + year)
      table_report[yi][0] = metric['vpxCoverage'][year]['min']
      table_report[yi][1] = metric['vpxCoverage'][year]['max']
      table_report[yi][2] = metric['vpxCoverage'][year]['gapPct']
      yi=yi+1
    pd_report = pd.DataFrame(table_report,index = table_index, columns = table_title)
    
    display(widgets.HTML(value='<br>The following table summarizes the valid pixels information.')) 
    display(pd_report)  
      
  return fgNbr
#getFitForPurpose(1)  
#@title Production Quality Indicators
def generate_production_and_validation_quality_indicators_report(metricId):
 
  lpMd=getLandProduct()
  if(lpMd != {}):
    metric=getMetric(metricId,lpMd)
    
    if (metric != {}):
      try:
        # urban_index=metric['classification']['codingClasses']['urban']
        # non_urban_index=metric['classification']['codingClasses']['non-urban']
        nb_classes=len(metric['classification']['codingClasses'])
       
        table_index = [''] * nb_classes
        table_title=[''] * (nb_classes+1)
       
        table_title[0]='Class / \\nIndicator'
   
        for i in range(nb_classes):
          table_title[i+1]='\\n'+metric['classification']['codingClasses'][i]
        middle_col=nb_classes//2+1
        table_title[middle_col]='REFERENCE'+table_title[middle_col]
        table_report=[ [0]*len(table_title) for i in range(nb_classes)]
      except KeyError:
         pass
      
      try:
        confusionMatrix=metric['classification']['confusionMatrix']
        for i in range(nb_classes):
          table_report[i][0]=metric['classification']['codingClasses'][i]
          for j in range (nb_classes):
            table_report[i][j+1]=confusionMatrix[i][j] 
        pd_report = pd.DataFrame(table_report ,index = table_index, columns = table_title)       
        pretty_print(pd_report,'')
        display(widgets.HTML(value='<br><br>'))
      except KeyError:
        pass  
      
 
 #    display other parameters
 #
      try:
        table_report=[
        ['Kappa',             getClassificationAttribute(metric['classification'],'kappa')],
        ['Overall Accuracy',  getClassificationAttribute(metric['classification'],'overallAccuracy')],   
        ['Producers Accuracy',getClassificationAttribute(metric['classification'],'producersAccuracy')],
        ['Users Accuracy',    getClassificationAttribute(metric['classification'],'usersAccuracy')]             
        ]
        table_index = ['','','','']
        table_title =['Other Parameters','Value [%]']
        pd_report = pd.DataFrame(table_report , index=table_index,columns = table_title)
        display(pd_report)
      except KeyError:
        pass 
      display(widgets.HTML(value='<br><br>'))
      try:
        displayValidationObject("Seasonality",metric['seasonality'] )
      except KeyError:
        pass

      try:
        displayValidationObject("Density Cover",metric['densityCover'] )
      except KeyError:
        pass


def displayValidationObject(title, theObject):
  try:
    table_report=[
                ['MAE',       theObject['mae']],
                ['RMSE',      theObject['rmse']], 
                ['MSE',       theObject['mse']],
                ['PearsonR',  theObject['pearsonR']],
                ['Coding Values',  toStringCodingValues(theObject['codingValues'])] 
      ]
    table_index = ['','','','','']
    table_title =[title,'Value']
    pd_report = pd.DataFrame(table_report , index=table_index,columns = table_title)
    pretty_print(pd_report,'')
  except KeyError:
    pass

def toStringCodingValues(codingValues):
  res=''
  for key in codingValues:
    res= res + key + ": "+ toString(codingValues[key]) + "\\n"
  return res

def  getMetricAttribute(metric,attribute):
  res=''
  try:
    res=metric[attribute]
  except KeyError:
    pass
  return res 


def generate_metadata_control_report(): 
  lpMd=getLandProduct()
  if(lpMd != {}):
    metric=getMetric("http://qcmms.esa.int/quality-indicators/#lpMetadataControlMetric",lpMd)
    if (metric != {}):
      try:
        metadataAvailable=getMetricAttribute(metric,'metadataAvailable')
        metadataCompliancy=getMetricAttribute(metric,'metadataCompliancy')
        metadataSpecification=getMetricAttribute(metric,'metadataSpecification')
        if (metadataAvailable):
          print("Land product metadata are available.")
          if metadataCompliancy:
            verb=" are "
          else:
            verb=" are not "
          print("Metadata"+verb+ "compliant with "+metadataSpecification + ".")
      except KeyError:
        pass  
def generate_product_ordinary_control_report():
  lpMd=getLandProduct()
  if(lpMd != {}):
    metric=getMetric("http://qcmms.esa.int/quality-indicators/#lpOrdinaryControlMetric",lpMd)
    if (metric != {}):
      #display(widgets.HTML(value='<br>'))
      table_report=[
          ['Lineage',      getMetricAttribute(metric,'lineage')],   
          ['Raster Format',getMetricAttribute(metric,'rasterFormat')],
          ['Readable',     getMetricAttribute(metric,'read')],
          ['Data Type',    getMetricAttribute(metric,'dataType')],
          ['EPSG',         getMetricAttribute(metric,'epsg')],
          ['Extent [ULX, ULY, LRX, LRY]',  getMetricAttribute(metric,'extentUlLr')], 
          ['Pixel Resolution in X direction',str(getMetricAttribute(metric,'xRes'))+'m'],  
          ['Pixel Resolution in Y direction',str(getMetricAttribute(metric,'yRes'))+'m'],  
          ['AOI Coverage Percentage', str(getMetricAttribute(metric,'aoiCoveragePct'))+'%']     
         ]
      table_index = ['','','','','','','','','']
      table_title = ['Parameters','Value']
      pd_report = pd.DataFrame(table_report , index=table_index,columns = table_title)
      display(pd_report)

def getClassificationAttribute(classification,attribute):
  res=''
  try:
      res=classification[attribute]
  except KeyError:
        pass
  return res 


def pretty_print(df, align):
  
  if align == '':
    #print(df.to_html().replace('\\n',"<br>"))
    return display(HTML(df.to_html().replace('\\n',"<br>")))
  else:
    print ("pretty_print")
    left_aligned_df = df.style.set_properties(**{'text-align': 'left'})
    left_aligned_df = left_aligned_df.set_table_styles(
        [dict(selector='th', props=[('text-align', 'left')])]
        )
    print( left_aligned_df.to_html().replace('\\n',"<br>"))
    return display( HTML(left_aligned_df.to_html().replace('\\n',"<br>")))

def displayIPsStatusBarchart():
  df = pd.DataFrame(IPsMetricsStatusVct)
  df.groupby(['Metric','Value']).sum().unstack().plot(kind='bar')
    
def cvNaFlag(flagApplicable):
  if not flagApplicable:
    return 'N/A'
  else:
    return ''

def cvExecutedFlag(statusObject):
  if not statusObject['Applicable']:
    return "N/A"
  elif statusObject['Executed']:
    return 'Executed'
  else:
    return 'Not Executed'

def cvLpStatus(statusObject):
  if not statusObject['Applicable']:
    return "N/A"
  elif not statusObject['Executed']:
    return 'Not Executed'
  elif statusObject['PASS']==1 :
    return 'PASS'
  else:
    return 'Failed'
    

def getAssessmentStatus():
  incomplete=False
  for i in range(len(MetricsStatusVct)):
    if MetricsStatusVct[i]['Applicable'] and not MetricsStatusVct[i]['Executed']:
      incomplete=True  
  if incomplete:
    return 'Incomplete'  
  if MetricsStatusVct[IndexLpMetadata]['PASS']==1 and MetricsStatusVct[IndexLpOrdinary]['PASS']==1 and MetricsStatusVct[IndexLpThematic]['PASS']==1:
    return 'PASS'
  else:
    return 'FAILED'

#generate_production_and_validation_quality_indicators_report("http://qcmms.esa.int/quality-indicators/#lpInterpretationMetric")
#@title Generate Overall Assessment
def generate_overall_assessment():
  table_title =['Task','Assessment', 'Status','Nb Pass','Nb Failed']
  table_index = ['','','','','','','','','','','']
  table_report = []
  lpMd = getLandProduct()
  nb_ip=countImageProductByMetric('','')
  if(lpMd != {}):
    
    table_report=[
      ['0.','Land Product Definitions','PASS','',''],
      ['1.','Production Feasibility',                cvExecutedFlag(MetricsStatusVct[IndexFeasibility]),
             MetricsStatusVct[IndexFeasibility]['PASS'],MetricsStatusVct[IndexFeasibility]['FAILED']],   
      ['2.','Delivery Control (images availability)',cvExecutedFlag(MetricsStatusVct[IndexDelivery]),
             MetricsStatusVct[IndexDelivery]['PASS'],MetricsStatusVct[IndexDelivery]['FAILED']],
      ['3A.','Ordinary Control (scene-level)', cvExecutedFlag(MetricsStatusVct[IndexOrdinary]),
             MetricsStatusVct[IndexOrdinary]['PASS'],MetricsStatusVct[IndexOrdinary]['FAILED']], 
      ['3B.','Detailed Control (pixel-level)', cvExecutedFlag(MetricsStatusVct[IndexDetailed]),
             MetricsStatusVct[IndexDetailed]['PASS'],MetricsStatusVct[IndexDetailed]['FAILED']], 
      ['4A.','Harmonization',                  cvExecutedFlag(MetricsStatusVct[IndexHarmonization]),
             MetricsStatusVct[IndexHarmonization]['PASS'],MetricsStatusVct[IndexHarmonization]['FAILED']],
      ['4B.','Fit-for-purpose',                cvLpStatus(MetricsStatusVct[IndexIpForLp]),         
             MetricsStatusVct[IndexIpForLp]['PASS'],MetricsStatusVct[IndexIpForLp]['FAILED']],  
      ['5.','Production Quality Indicators',   cvLpStatus(MetricsStatusVct[IndexLpInterpretation]),
             MetricsStatusVct[IndexLpInterpretation]['PASS'],MetricsStatusVct[IndexLpInterpretation]['FAILED']],
      ['6A.','Metadata Control Indicators',    cvLpStatus(MetricsStatusVct[IndexLpMetadata]),
             MetricsStatusVct[IndexLpMetadata]['PASS'],MetricsStatusVct[IndexLpMetadata]['FAILED']],
      ['6B.','Ordinary Control Indicators',    cvLpStatus(MetricsStatusVct[IndexLpOrdinary]),      
             MetricsStatusVct[IndexLpOrdinary]['PASS'],MetricsStatusVct[IndexLpOrdinary]['FAILED']],
      ['6C.','Thematic Validation Indicators', cvLpStatus(MetricsStatusVct[IndexLpThematic]),      
             MetricsStatusVct[IndexLpThematic]['PASS'],MetricsStatusVct[IndexLpThematic]['FAILED']]          
    ]
    
    pd_report = pd.DataFrame(table_report , index=table_index,columns = table_title)
    display(pd_report)

     
#@title Interpretation Form (not used)
#
# Store and retrieve in a file (JSON object stored as a string in a file)
# the text entered by the user in the all textarea associated to the different tables/figures of the report
interpretationFile = 'interpretations.json'
interpretationList=['feasibility','cloudPercentages','delivery','ordinary','detailed','detailedStat','geometry']
#  object that with attributes above and text read and entered by the user
interpretationTexts={}
# object that contains for each textarea the associated widget object
interpretationWdgts={}
defaultText=''
wdgtPlaceHolder='Enter your interpretation for '

def initInterpretationTexts():
# read interpretation comment from a file located in the same place than the notebook
  interpretations={}
  try:
    with open(interpretationFile) as json_file:
      interpretations = json.load(json_file)
#     check that different text areas had been save. Initialize new textareas (update report definition).
      for i in range(len(interpretationList)):
        try:
          interpretationName=interpretations[interpretationList[i]]
        except KeyError:
          interpretations[interpretationName]=defaultText 
  except Exception as e:
    for i in range(len(interpretationList)):
      interpretationName=interpretationList[i]
      interpretations[interpretationName]=defaultText
  return interpretations

# read from file
interpretationTexts=initInterpretationTexts()

# create one text area to enter interpretation comments
def createTextArea(interpretationName):
  wdgtNote=widgets.HTML(
    value="<b><i>Interpretation note:</i></b>"
    )
  wdgt=widgets.Textarea(
    value=interpretationTexts[interpretationName],
    placeholder=wdgtPlaceHolder + interpretationName,
    disabled=False,
    layout=widgets.Layout(width='800px', height='80px'))
  interpretationWdgts[interpretationName]=wdgt
  wdgtAll=widgets.VBox((wdgtNote,wdgt))
  display(wdgtAll)

# display the button to save the entered interpretation texts in the file.
def createSaveButton():
  button = widgets.Button(description='Save your inputs',background_color='gray',layout=widgets.Layout(width='200px'))
  display(button)
  button.on_click(on_click)
# called back activated when the button is clicked
# save the interpretations in a file.
def on_click(b):
  for i in range(len(interpretationList)):
    interpretationTexts[interpretationList[i]]=interpretationWdgts[interpretationList[i]].value
  with open(interpretationFile,'w') as outfile:
    json.dump(interpretationTexts, outfile)
  display(interpretationTexts)  
#@title Default title text
#lpMd=getLandProduct()
#ipUrl=initImageProductsUrl()
#platforms=initPlatforms(lpMd)
#@title Generate Report Title
def generateReportTitle(lpMd):
  title=lpMd['properties']['title']
  productFocus=lpMd['properties']['additionalAttributes']['product_focus']
  display(widgets.HTML(
  value=  
  '<br><font size="18"><center>Mapping Production</center></font>' +
  '<br><font size="20"><center>QUALITY ASSESSMENT REPORT</center></font>' +
  '<br><font size="16"><center>Demo: '+ title+'</center><font>'
  ))

generateReportTitle(lpMd)














# 1. Introduction



This document is the QCMMS Quality Assessment Report based
on collected quality-related metadata by QC-Manager, stored in Catalog
and analysed by QA-Report manager. The report

-   provides insights about the quality of the whole mapping process (based on simplified analytic information);

-   informs the service user, and the service provider, about the potential quality issues at each stage during the manufacturing of the land product;

-   guides the reader to trace the error / uncertainty source from the report and catalogued metadata towards the critical process;

-   and provides final quality assement about the mapped product (delivery: Yes / No).








# 2. Applicable and Reference Documents
* [tsi] Vermote et al. (2009): Temporal Smoothness Index
* [tc16Sd] Qiu et al. (2019): Temporal Consistency


# 3. Terms, Definitions And Abbreviated Terms

## 3.1 Definitions


In [ ]:
generateDefinitions()


## 3.2 Abbreviated Terms

In [ ]:
generateAbbreviations()

# 4. Processing Report

## 4.1 Land Product Definitions

This section provides information about the land product definition and technical specifications.

In [ ]:
#@title
res=generate_lp_definition_report(tableNbr)
tableNbr=res['tableNbr']
geometric_resolution=res['geometric_resolution']

In the previous table, the "Thematic accuracy" is the percentage of user's / producer's accuracy (sealing levels > 20-30%). 

## 4.2 Production Feasibility Control

This section computes the number of input products that pass or not pass a quality check among the set of products selected to build the land product. 

These quality checks are extracted from the product metadata retrieved in the archive.  


In [ ]:
#@title
if alreadyExecuted(IndexFeasibility, True):
  for i in range(len(platforms)): 
     tableNbr=generateTableLabel(tableNbr,'General IP quality metadata overview as retrieved from archive for ' + platforms[i])
     generate_feasibility_control_report(platforms[i])
#createTextArea('feasibility')


The following graph shows on y-axis the number of images with a cloud cover percentage provided on x-axis.

In [ ]:
#@title 
if alreadyExecuted(IndexFeasibility, True):
  for i in range(len(platforms)): 
     fgNbr=generateHist(fgNbr,platforms[i])

## 4.3 Delivery Control

The following table provides information about the download status of the products used to build the Land Product.

  



In [ ]:
#@title 
if alreadyExecuted(IndexDelivery, True): 
    tableNbr=generateTableLabel(tableNbr,'Tabulated overview of the delivery control')
    generate_delivery_control_report(platforms)
#createTextArea('delivery')

Information about the table content:  
*   Column "TRUE":  number of products that could be downloaded for each of the platforms and then the total for all platforms
*   Column "FALSE":  number of products that could not be downloaded for each of the platforms and then the total for all platforms
*   Column "WAITING":  number of products for which the delivery process is in WAITING state for each of the platforms and then the total for all platforms
*   Column "IN_PROGRESS":  number of products for which the delivery process is in progress for each of the platforms and then the total for all platforms.

## 4.4 Ordinary Control

The ordinary control provides information about quality checks done at the level of the input products raster related metadata for the different levels of processings. 

In [ ]:
#@title
if alreadyExecuted(IndexOrdinary, True):
  for i in range(len(platforms)):
    tableNbr=generateTableLabel(tableNbr,'Tabulated overview of the L1 & L2 products ordinary control for '+ platforms[i])
    generate_ordinary_control_report(platforms[i])
#createTextArea('ordinary')

Information about the table content:  
*   Rasters complete:  number of products with complete rasters
*   Rasters read: number of products that pass the test to read the respective image band 
*   Calibration metadata: number of products that pass the calibration metadata availability checks
*   Metadata read: number of products that pass the metadata check
*   Total: number of products which pass or fail this "ordinary check" and include "ordinary check" quality metadata for the corresponding processing level.

## 4.5 Detailed Control

This control is focused on detailed pixel-level quality metadata. For each input image, it generates a mask for different topics (cloud coverage, valid pixels, geometry). <br>The following table provides  the number of input products for which a mask has been generated in both cases: case where the detailed control status is "PASS" or "FAILED". <br> The "Valid pixel metadata" provides summary information about the usability of individual pixels within each scene that passed the previous quality assessment procedure.<br>
The "Total" column indicates the number of input products that pass or not this step.

In [ ]:
#@title 
if alreadyExecuted(IndexDetailed, True):
  for i in range(len(platforms)): 
    tableNbr=generateTableLabel(tableNbr,'Availability of the detailed pixel level quality metadata for '+ platforms[i])
    generate_detailed_control_report(platforms[i])

Information about the table content:  
*   Cloud cover layer:  number of products for which a cloud cover mask has been generated (row "true") or not generated (row "false")
*   Cloud cover alternative layer: number of products for which an alternative cloud cover mask  has been generated (row "true") or not generated (row "false") using the Fmask algorithm
*   Geometry val.layer: number of products for which a geometry control spatial  mask  has been generated (row "true") or not generated (row "false")
*   Valid pixel metadata: number of products for which a valid pixels spatial   mask  has been generated (row "true") or not generated (row "false")
*   Total: number of products which pass or fail this "detailed control" 

The following table provides statistics (mean) on cloud coverage, geometry quality parameters computed on the set of input products. It also provides the number of these products for which the maximum difference on X and Y is greater than the land product required geometry  accuracy. The geometry is evaluated against a reference spatial layer.

In [ ]:
#@title 
if alreadyExecuted(IndexDetailed, True):
  for i in range(len(platforms)): 
    tableNbr=generateTableLabel(tableNbr,'Summary statistics of the pixel level quality metadata for '+ platforms[i])
    generate_detailed_control_stat_report(platforms[i])

Information about the table content:  
*   Mean cloud cover:  mean of cloud cover percentage of the input images based on the cloud cover mask delivered with the image products
*   Mean cloud  alternative: same but computed on the alternative cloud cover mask
*   Mean geometry RmseX: mean of the root mean square error (X-axis) of the input images. In the input images, the error is computed by comparison with geometric reference in meters.
*   Mean geometry RmseY: same on Y-axis
*   Nb > d Geometry diffX Max: number of products for which the maximum value (X-axis) of the difference between IP and geometry reference is greater than d in meter.
*   Nb > d Geometry DiffY Max: same on Y-axis 
*   Mean valid pixels: mean of the valids pixels percentage of the input images. 

The following picture is a plot of the DiffX, DiffY values from the input images. The displayed circle radius has been computed to include 90% of these points. DiffX is the maximum value (X-axis) of the
difference between the image product and the geometry reference (value in meters). same for DiffY on Y-axis.

In [ ]:
#@title 
if alreadyExecuted(IndexDetailed, True):
  for i in range(len(platforms)):
    #tableNbr=generateTableLabel(tableNbr,'Tabulated overview of the L1 & L2 products ordinary control for '+ platforms[i])
    #print(platforms[i])
    fgNbr=generate_Plot_CE(fgNbr,platforms[i])

## 4.6 Harmonization Control

In [ ]:
#@title 
if alreadyExecuted(IndexHarmonization, True):
  print("The following graphics show the values of temporal indicators computed on each band of the input products.")
  print("")
  for i in range(len(platforms)):
    res=displayRadiometryConsistencyBarchart(fgNbr,platforms[i])
    fgNbr=res['fgNbr']
    references=res['references']

In [ ]:
#@title 
if alreadyExecuted(IndexHarmonization, False):
  print("References:")
  for i in range(len(references)): 
    print(references[i]['category'] +": " + references[i]['title'] ) 

In [ ]:
from IPython.display import display, Math, Latex
if alreadyExecuted(IndexHarmonization, False):

  print("Additional information:")
  print("tc16Sd computes temporal consistency indicator for each band.") 
  print("The Standard Deviation (Sd) of L2 Surface Reflectance difference is chosen by Qiu et al. (2019) to quantify the consistency of the whole time series.")
  print('The image product spectral reflectance (*10^4) for each band is used as the inputs to the analysis.')
  print('Scaling factor is 0.0001 for the representation of the surface reflectance.') 
  print("")
  print("tsiCE90 computes a Temporal Smoothness Index (TSI) for each band.")
  print("")
  display(Math(r'NoiseR(y)=\frac{1}{\sqrt{N-2}}* \sqrt{\sum_{i=1}^{n-2}(\frac{y_{i+1}-\frac{y_{i+2}-y_{i}}{day_{i+2}-day_i}(day_{i+1}-day_i)-y_i}  {y_{i+1}})^2}'))
  print("")
  print("This TSI estimate proposed by Vermote et al. (2009) assumes a linear variation between two dates, a few days apart (proposed 20).")
  print('Given three successive measurements, a triplet (i, i + 1, and i + 2), the statistical difference between the center measurement and the linear interpolation between the two extremes quantifies the noise.')
  print("y represents the band spectral value and N is the number of input products in the analysis.")
 


## 4.7 Fit-for-Purpose Control

The following images present aggregations from valid pixels metadata over N image products computed by year (red= gaps). 
The possible status values of this step are:
*  "NONE": there is no single pixel with good quality spectral value
*  "PARTIAL": nonzero percentage gap up to gap of 5%  
*  "FULL": there is no gap.


In [ ]:
#@title 
if alreadyExecuted(IndexIpForLp, True):
  fgNbr=getFitForPurpose(fgNbr)

Information about the table content:  
*   Column "Min":  minimum value in the vpxCoverage layer indicating how many repeated cloud free observations were identified for given one individual pixel 
*   Column "Max": maximum value in the vpxCoverage layer indicating how many repeated cloud free observations were identified for given one individual pixel
*   Gap percentage: percentage of pixels with zero valid
pixel value in the set of IPs.

## 4.8 Production Quality Indicators



In [ ]:
#@title 
if alreadyExecuted(IndexLpInterpretation, True):
  tableNbr=generateTableLabel(tableNbr,'Interpretation model training indicators for Urban mask')
  generate_production_and_validation_quality_indicators_report("http://qcmms.esa.int/quality-indicators/#lpInterpretationMetric")


## 4.9 Land Product Validation Quality Indicators

This section provides information on the Thematic validation of the Land Product.  It provides 'standard' Land Product metadata information about how good the resulting Land Product is, compared to available reference.

See section 3.1 for the definition of these parameters.

In [ ]:



#@title 
if alreadyExecuted(IndexLpThematic, True):
  tableNbr=generateTableLabel(tableNbr,'Land product independent validation indicators')
  generate_production_and_validation_quality_indicators_report("http://qcmms.esa.int/quality-indicators/#lpThematicValidationMetric")

## 4.10 Land Product Metadata Control 



In [ ]:
if alreadyExecuted(IndexLpMetadata, True):
  generate_metadata_control_report()

## 4.11 Land Product Ordinary Control 

 The land product ordinary control provides information on the land product to evaluate the basic Land Product characteristics and its compliancy to initial requirements.

In [ ]:
if alreadyExecuted(IndexLpOrdinary, True):
  tableNbr=generateTableLabel(tableNbr,'Land product ordinary control parameters')
  generate_product_ordinary_control_report()

## 4.12 Overall Assessment

This section provides an overall assessment of the quality analysis. 

The following bar chart shows the number of image products that could pass or fail the quality check associated to a processing step.

In [ ]:
#@title
tableNbr=generateTableLabel(tableNbr,'Metric values for Image Products')
displayIPsStatusBarchart()


The following table gives the number of products that pass or fail the quality check associated to a processing step.  Steps 1 to 4A provides the number of input products. The other steps concern the current land product being produced.

In [ ]:
tableNbr=generateTableLabel(tableNbr,'Overall Assessment')
generate_overall_assessment()